---
# 1. Teoria: Information Retrieval

## 1.1 Definizione e Problema

**Information Retrieval (IR)** è la disciplina che si occupa di trovare materiale (solitamente documenti) di natura non strutturata (solitamente testo) che soddisfa un bisogno informativo, all'interno di grandi collezioni.

### Il Problema Fondamentale

```
QUERY UTENTE  →  SISTEMA IR  →  DOCUMENTI RILEVANTI (ranked)
   "AI for healthcare"     ?       [doc1, doc5, doc12, ...]
```

**Sfide principali:**

1. **Vocabulary mismatch**: utente usa "car", documento contiene "automobile"
2. **Ambiguità**: "bank" = banca finanziaria o riva del fiume?
3. **Rilevanza soggettiva**: ciò che è rilevante dipende dal contesto
4. **Scalabilità**: milioni/miliardi di documenti

---

## 1.2 Modelli di Information Retrieval

### Modello Booleano

Il più semplice: documenti sono insiemi di termini, query sono espressioni booleane.

```
Query: "python AND machine learning AND NOT java"
```

**Vantaggi:** Preciso, deterministico  
**Svantaggi:** Nessun ranking, troppo rigido, difficile per utenti

### Modello Vettoriale (Vector Space Model)

Documenti e query sono rappresentati come **vettori** in uno spazio n-dimensionale, dove n = numero di termini nel vocabolario.

```
Vocabolario: [python, data, machine, learning, sql]

Doc1: "python data analysis"   → [1, 1, 0, 0, 0]  (o con TF-IDF)
Doc2: "machine learning python" → [1, 0, 1, 1, 0]
Query: "python machine learning" → [1, 0, 1, 1, 0]
```

**Similarità** tra query e documento = similarità tra vettori.

### Similarità Coseno

La misura più usata per confrontare vettori in IR:

$$\text{cosine}(\vec{q}, \vec{d}) = \frac{\vec{q} \cdot \vec{d}}{||\vec{q}|| \times ||\vec{d}||}$$

- Range: [0, 1] per vettori non-negativi (TF-IDF)
- 1 = identici, 0 = ortogonali (nulla in comune)
- Invariante rispetto alla lunghezza del documento

---

## 1.3 Indice Invertito

Struttura dati fondamentale per IR efficiente.

**Forward Index** (inefficiente per ricerca):
```
Doc1 → [python, data, analysis]
Doc2 → [machine, learning, python]
Doc3 → [sql, database, python]
```

**Inverted Index** (efficiente):
```
python    → [Doc1, Doc2, Doc3]
data      → [Doc1]
machine   → [Doc2]
learning  → [Doc2]
sql       → [Doc3]
database  → [Doc3]
analysis  → [Doc1]
```

**Con posizioni** (per phrase queries):
```
python → [Doc1:pos1, Doc2:pos3, Doc3:pos3]
```

---

## 1.4 Valutazione dei Sistemi IR

### Precision e Recall

Per una query, dato un insieme di documenti **rilevanti** (ground truth):

$$\text{Precision} = \frac{|\text{Retrieved} \cap \text{Relevant}|}{|\text{Retrieved}|}$$

$$\text{Recall} = \frac{|\text{Retrieved} \cap \text{Relevant}|}{|\text{Relevant}|}$$

**Esempio:**
- Documenti totali: 100
- Documenti rilevanti: 10
- Documenti restituiti dal sistema: 8
- Di questi, 5 sono effettivamente rilevanti

```
Precision = 5/8 = 62.5%  ("quanto è pulito il risultato")
Recall    = 5/10 = 50%   ("quanto ho trovato del totale")
```

### F1-Score

Media armonica di precision e recall:

$$F_1 = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}$$

### Precision@K e Recall@K

Valuta solo i primi K risultati:

```
Precision@5 = rilevanti nei primi 5 / 5
Precision@10 = rilevanti nei primi 10 / 10
```

### Mean Average Precision (MAP)

Media delle Average Precision su più query:

$$AP(q) = \frac{1}{|R_q|} \sum_{k=1}^{n} P(k) \cdot rel(k)$$

Dove:
- $P(k)$ = precision at rank k
- $rel(k)$ = 1 se doc a rank k è rilevante, 0 altrimenti

---

## 1.5 Tecniche Avanzate

### Query Expansion

Aggiunge termini alla query per migliorare recall:

```
Query originale: "automobile"
Query espansa:   "automobile car vehicle auto"
```

**Tecniche:**
- Sinonimi da thesaurus
- Termini da documenti rilevanti (relevance feedback)
- Word embeddings (termini simili nello spazio vettoriale)

### Relevance Feedback

L'utente indica quali risultati sono rilevanti, il sistema riformula la query.

**Rocchio Algorithm:**

$$\vec{q}_{new} = \alpha \vec{q}_{old} + \beta \frac{1}{|D_r|} \sum_{d \in D_r} \vec{d} - \gamma \frac{1}{|D_{nr}|} \sum_{d \in D_{nr}} \vec{d}$$

Dove:
- $D_r$ = documenti rilevanti
- $D_{nr}$ = documenti non rilevanti
- $\alpha, \beta, \gamma$ = pesi (tipicamente 1, 0.75, 0.15)

---
# 2. Schema Mentale

## Architettura Sistema IR

```
                    ┌─────────────────────────────────────┐
                    │         SISTEMA IR                   │
                    └─────────────────────────────────────┘
                                    │
          ┌─────────────────────────┼─────────────────────────┐
          │                         │                         │
          ▼                         ▼                         ▼
  ┌───────────────┐        ┌───────────────┐        ┌───────────────┐
  │   INDEXING    │        │   RETRIEVAL   │        │  EVALUATION   │
  │               │        │               │        │               │
  │ • Tokenize    │        │ • Parse query │        │ • Precision   │
  │ • Normalize   │        │ • Vectorize   │        │ • Recall      │
  │ • Inverted Idx│        │ • Similarity  │        │ • F1          │
  │ • TF-IDF      │        │ • Rank        │        │ • MAP         │
  └───────────────┘        └───────────────┘        └───────────────┘
          │                         │                         │
          ▼                         ▼                         ▼
   ┌────────────┐           ┌────────────┐           ┌────────────┐
   │   INDEX    │           │  RANKED    │           │  METRICS   │
   │  (stored)  │──────────▶│  RESULTS   │──────────▶│  REPORT    │
   └────────────┘           └────────────┘           └────────────┘
```

## Decision Tree: Quale Approccio?

```
                    Tipo di Ricerca?
                          │
        ┌─────────────────┼─────────────────┐
        │                 │                 │
        ▼                 ▼                 ▼
     Esatta          Semantica         Ibrida
        │                 │                 │
        ▼                 ▼                 ▼
   Inverted Index    Embeddings      BM25 + Dense
   + BM25/TF-IDF     + ANN Search    Retrieval
```

## Pipeline Implementativa

```
DOCUMENTI ──▶ Preprocessing ──▶ Vectorization ──▶ INDEX
                   │                  │              │
                   │                  │              │
              Lowercase          TF-IDF           Store
              Tokenize           Matrice          (pickle/
              Stopwords          Sparsa            disk)
              Stemming


QUERY ──▶ Same Preprocessing ──▶ Vectorization ──▶ SIMILARITY ──▶ RANKED DOCS
                                                        │
                                                        ▼
                                                  Cosine(q, d)
                                                  for each d
```

## Metriche: Quando Usare Cosa

| Metrica | Usa Quando | Interpretazione |
|---------|------------|------------------|
| Precision@K | Utente guarda pochi risultati | Qualità top results |
| Recall | Trovare tutto è importante | Copertura |
| F1 | Bilanciare P e R | Score unico |
| MAP | Confrontare sistemi | Performance media |
| MRR | Un solo doc rilevante | Rank del primo hit |

---
# 3. Notebook Dimostrativo

In [ ]:
# ============================================================
# SETUP E IMPORT
# ============================================================

import numpy as np
import re
from typing import List, Dict, Tuple, Optional
from collections import defaultdict
from dataclasses import dataclass
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

print("Import completati.")
print(f"NumPy version: {np.__version__}")

In [ ]:
# ============================================================
# DEMO 1: INDICE INVERTITO
# ============================================================

class InvertedIndex:
    """
    Implementazione base di un indice invertito.
    
    L'indice mappa ogni termine ai documenti che lo contengono,
    con le posizioni di occorrenza.
    """
    
    def __init__(self):
        # Struttura: {termine: {doc_id: [posizioni]}}
        self.index = defaultdict(lambda: defaultdict(list))
        # Documenti originali
        self.documents = {}
        # Statistiche
        self.doc_count = 0
        self.term_count = 0
    
    def tokenize(self, text: str) -> List[str]:
        """Tokenizzazione semplice: lowercase + split su non-alfanumerici."""
        text = text.lower()
        tokens = re.findall(r'\b[a-z]+\b', text)  # Solo parole alfabetiche
        return tokens
    
    def add_document(self, doc_id: str, text: str):
        """Aggiunge un documento all'indice."""
        # Salva documento originale
        self.documents[doc_id] = text
        self.doc_count += 1
        
        # Tokenizza e indicizza
        tokens = self.tokenize(text)
        for position, token in enumerate(tokens):
            self.index[token][doc_id].append(position)
        
        # Aggiorna conteggio termini unici
        self.term_count = len(self.index)
    
    def search(self, query: str) -> Dict[str, List[int]]:
        """Cerca documenti che contengono il termine."""
        query_term = query.lower().strip()
        if query_term in self.index:
            return dict(self.index[query_term])
        return {}
    
    def boolean_search(self, terms: List[str], operator: str = 'AND') -> List[str]:
        """Ricerca booleana: AND o OR tra termini."""
        if not terms:
            return []
        
        # Trova documenti per ogni termine
        doc_sets = []
        for term in terms:
            term_lower = term.lower().strip()
            if term_lower in self.index:
                doc_sets.append(set(self.index[term_lower].keys()))
            else:
                doc_sets.append(set())
        
        # Applica operatore
        if operator.upper() == 'AND':
            result = doc_sets[0]
            for s in doc_sets[1:]:
                result = result.intersection(s)
        else:  # OR
            result = set()
            for s in doc_sets:
                result = result.union(s)
        
        return list(result)
    
    def get_stats(self) -> Dict:
        """Statistiche sull'indice."""
        return {
            'num_documents': self.doc_count,
            'vocabulary_size': self.term_count,
            'avg_terms_per_doc': sum(len(self.tokenize(d)) 
                                     for d in self.documents.values()) / max(self.doc_count, 1)
        }

# Test dell'indice invertito
print("="*60)
print("DEMO 1: INDICE INVERTITO")
print("="*60)

# Crea indice
idx = InvertedIndex()

# Aggiungi documenti
docs = {
    'doc1': 'Python is great for data science and machine learning',
    'doc2': 'Machine learning with Python and TensorFlow',
    'doc3': 'Data analysis using SQL and Python',
    'doc4': 'JavaScript for web development',
    'doc5': 'Deep learning and neural networks with Python'
}

for doc_id, text in docs.items():
    idx.add_document(doc_id, text)

print("\n📊 Statistiche indice:")
stats = idx.get_stats()
for k, v in stats.items():
    print(f"   {k}: {v:.2f}" if isinstance(v, float) else f"   {k}: {v}")

# Ricerca singolo termine
print("\n🔍 Ricerca 'python':")
result = idx.search('python')
for doc_id, positions in result.items():
    print(f"   {doc_id}: posizioni {positions}")

# Ricerca booleana AND
print("\n🔍 Ricerca booleana 'python AND learning':")
result_and = idx.boolean_search(['python', 'learning'], 'AND')
print(f"   Documenti: {result_and}")

# Ricerca booleana OR
print("\n🔍 Ricerca booleana 'sql OR javascript':")
result_or = idx.boolean_search(['sql', 'javascript'], 'OR')
print(f"   Documenti: {result_or}")

In [ ]:
# ============================================================
# DEMO 2: SISTEMA IR CON TF-IDF E COSINE SIMILARITY
# ============================================================

@dataclass
class SearchResult:
    """Risultato di una ricerca."""
    doc_id: str
    score: float
    snippet: str

class TFIDFSearchEngine:
    """
    Motore di ricerca basato su TF-IDF e similarità coseno.
    
    Workflow:
    1. Indicizza documenti con TF-IDF
    2. Trasforma query nello stesso spazio
    3. Calcola similarità coseno
    4. Restituisce documenti ranked
    """
    
    def __init__(self, max_features: int = 5000):
        self.vectorizer = TfidfVectorizer(
            max_features=max_features,  # Limita vocabolario
            stop_words='english',       # Rimuovi stopwords
            ngram_range=(1, 2),         # Unigrammi e bigrammi
            lowercase=True
        )
        self.doc_vectors = None    # Matrice TF-IDF documenti
        self.documents = {}         # doc_id -> testo
        self.doc_ids = []           # Lista ordinata doc_id
        self.is_fitted = False
    
    def index(self, documents: Dict[str, str]):
        """
        Indicizza una collezione di documenti.
        
        Args:
            documents: {doc_id: testo}
        """
        self.documents = documents
        self.doc_ids = list(documents.keys())
        
        # Crea matrice TF-IDF
        texts = [documents[doc_id] for doc_id in self.doc_ids]
        self.doc_vectors = self.vectorizer.fit_transform(texts)
        self.is_fitted = True
        
        print(f"Indicizzati {len(documents)} documenti")
        print(f"Vocabolario: {len(self.vectorizer.vocabulary_)} termini")
        print(f"Matrice: {self.doc_vectors.shape}")
    
    def search(self, query: str, top_k: int = 5) -> List[SearchResult]:
        """
        Cerca documenti rilevanti per la query.
        
        Args:
            query: testo della query
            top_k: numero massimo di risultati
            
        Returns:
            Lista di SearchResult ordinati per score decrescente
        """
        if not self.is_fitted:
            raise ValueError("Indice non inizializzato. Chiamare index() prima.")
        
        # Trasforma query nello stesso spazio TF-IDF
        query_vector = self.vectorizer.transform([query])
        
        # Calcola similarità coseno con tutti i documenti
        similarities = cosine_similarity(query_vector, self.doc_vectors)[0]
        
        # Ordina per similarità decrescente
        ranked_indices = similarities.argsort()[::-1][:top_k]
        
        # Costruisci risultati
        results = []
        for idx in ranked_indices:
            if similarities[idx] > 0:  # Solo risultati con match
                doc_id = self.doc_ids[idx]
                results.append(SearchResult(
                    doc_id=doc_id,
                    score=similarities[idx],
                    snippet=self.documents[doc_id][:100] + '...'
                ))
        
        return results
    
    def get_similar_docs(self, doc_id: str, top_k: int = 3) -> List[SearchResult]:
        """Trova documenti simili a un documento dato."""
        if doc_id not in self.documents:
            raise ValueError(f"Documento {doc_id} non trovato")
        
        # Usa il documento come query
        return self.search(self.documents[doc_id], top_k + 1)[1:]  # Escludi se stesso

# Test
print("="*60)
print("DEMO 2: SEARCH ENGINE TF-IDF")
print("="*60)

# Dataset di documenti (articoli fittizi)
articles = {
    'art1': 'Python programming language is widely used in data science. Python offers many libraries for machine learning and data analysis.',
    'art2': 'Machine learning algorithms can predict outcomes based on data. Deep learning is a subset of machine learning using neural networks.',
    'art3': 'SQL databases store structured data efficiently. Query optimization is important for database performance.',
    'art4': 'Web development with JavaScript and React creates interactive user interfaces. Frontend frameworks improve developer productivity.',
    'art5': 'Data visualization helps communicate insights. Charts and graphs make data more understandable for stakeholders.',
    'art6': 'Natural language processing enables computers to understand human language. Text classification and sentiment analysis are NLP applications.',
    'art7': 'Cloud computing provides scalable infrastructure. AWS and Azure offer machine learning services in the cloud.',
    'art8': 'Statistical analysis reveals patterns in data. Hypothesis testing and regression are fundamental statistical methods.'
}

# Crea e popola search engine
engine = TFIDFSearchEngine()
engine.index(articles)

# Test ricerche
queries = [
    'machine learning python',
    'database SQL query',
    'data visualization charts'
]

for query in queries:
    print(f"\n🔍 Query: '{query}'")
    print("-" * 50)
    results = engine.search(query, top_k=3)
    for i, res in enumerate(results, 1):
        print(f"   {i}. [{res.doc_id}] Score: {res.score:.4f}")
        print(f"      {res.snippet}")

# Documenti simili
print(f"\n{'='*60}")
print("DOCUMENTI SIMILI")
print("="*60)
print("\n📄 Documenti simili a 'art1' (Python programming):")
similar = engine.get_similar_docs('art1', top_k=2)
for res in similar:
    print(f"   - [{res.doc_id}] Score: {res.score:.4f}")

In [ ]:
# ============================================================
# DEMO 3: VALUTAZIONE IR - PRECISION, RECALL, F1
# ============================================================

class IRMetrics:
    """
    Calcola metriche di valutazione per sistemi IR.
    
    Richiede:
    - Documenti restituiti dal sistema (retrieved)
    - Documenti rilevanti secondo ground truth (relevant)
    """
    
    @staticmethod
    def precision(retrieved: List[str], relevant: set) -> float:
        """Precision = |retrieved ∩ relevant| / |retrieved|"""
        if not retrieved:
            return 0.0
        retrieved_set = set(retrieved)
        return len(retrieved_set & relevant) / len(retrieved_set)
    
    @staticmethod
    def recall(retrieved: List[str], relevant: set) -> float:
        """Recall = |retrieved ∩ relevant| / |relevant|"""
        if not relevant:
            return 0.0
        retrieved_set = set(retrieved)
        return len(retrieved_set & relevant) / len(relevant)
    
    @staticmethod
    def f1_score(precision: float, recall: float) -> float:
        """F1 = 2 * (precision * recall) / (precision + recall)"""
        if precision + recall == 0:
            return 0.0
        return 2 * (precision * recall) / (precision + recall)
    
    @staticmethod
    def precision_at_k(retrieved: List[str], relevant: set, k: int) -> float:
        """Precision considerando solo i primi k risultati."""
        return IRMetrics.precision(retrieved[:k], relevant)
    
    @staticmethod
    def average_precision(retrieved: List[str], relevant: set) -> float:
        """
        Average Precision: media delle precision a ogni rank dove
        compare un documento rilevante.
        """
        if not relevant:
            return 0.0
        
        precisions = []
        relevant_found = 0
        
        for i, doc in enumerate(retrieved):
            if doc in relevant:
                relevant_found += 1
                p_at_i = relevant_found / (i + 1)
                precisions.append(p_at_i)
        
        if not precisions:
            return 0.0
        
        return sum(precisions) / len(relevant)
    
    @staticmethod
    def mean_average_precision(queries_results: List[Tuple[List[str], set]]) -> float:
        """
        MAP: media delle Average Precision su multiple query.
        
        Args:
            queries_results: [(retrieved_list, relevant_set), ...]
        """
        if not queries_results:
            return 0.0
        
        aps = [IRMetrics.average_precision(r, rel) for r, rel in queries_results]
        return sum(aps) / len(aps)

# Test metriche
print("="*60)
print("DEMO 3: METRICHE IR")
print("="*60)

# Scenario: ricerca "machine learning"
# Ground truth: documenti effettivamente rilevanti
relevant_docs = {'art1', 'art2', 'art6', 'art7'}  # 4 documenti rilevanti

# Sistema restituisce (in ordine di ranking)
retrieved_docs = ['art2', 'art1', 'art5', 'art6', 'art3']  # 5 risultati

print("\n📊 Scenario di valutazione")
print(f"   Documenti rilevanti (ground truth): {relevant_docs}")
print(f"   Documenti restituiti (ranked): {retrieved_docs}")

# Calcola metriche
metrics = IRMetrics()

p = metrics.precision(retrieved_docs, relevant_docs)
r = metrics.recall(retrieved_docs, relevant_docs)
f1 = metrics.f1_score(p, r)

print(f"\n📈 Metriche Base:")
print(f"   Precision: {p:.2%} ({len(set(retrieved_docs) & relevant_docs)}/{len(retrieved_docs)})")
print(f"   Recall:    {r:.2%} ({len(set(retrieved_docs) & relevant_docs)}/{len(relevant_docs)})")
print(f"   F1-Score:  {f1:.2%}")

# Precision@K
print(f"\n📈 Precision@K:")
for k in [1, 3, 5]:
    p_at_k = metrics.precision_at_k(retrieved_docs, relevant_docs, k)
    print(f"   P@{k}: {p_at_k:.2%}")

# Average Precision
ap = metrics.average_precision(retrieved_docs, relevant_docs)
print(f"\n📈 Average Precision: {ap:.4f}")

# Spiegazione dettagliata AP
print("\n   Calcolo AP step by step:")
rel_count = 0
for i, doc in enumerate(retrieved_docs):
    is_rel = "✓" if doc in relevant_docs else "✗"
    if doc in relevant_docs:
        rel_count += 1
        p_at_i = rel_count / (i + 1)
        print(f"   Rank {i+1}: {doc} {is_rel} → P@{i+1}={p_at_i:.3f} (contribuisce ad AP)")
    else:
        print(f"   Rank {i+1}: {doc} {is_rel}")

In [ ]:
# ============================================================
# DEMO 4: QUERY EXPANSION
# ============================================================

class QueryExpander:
    """
    Espande query con sinonimi e termini correlati.
    
    Tecniche:
    1. Thesaurus-based: sinonimi da dizionario
    2. Corpus-based: termini co-occorrenti
    """
    
    def __init__(self):
        # Thesaurus semplificato
        self.synonyms = {
            'car': ['automobile', 'vehicle', 'auto'],
            'big': ['large', 'huge', 'enormous'],
            'fast': ['quick', 'rapid', 'speedy'],
            'machine learning': ['ml', 'artificial intelligence', 'ai'],
            'database': ['db', 'data store', 'databank'],
            'error': ['bug', 'fault', 'defect', 'issue'],
            'analyze': ['examine', 'investigate', 'study'],
            'python': ['py'],
            'javascript': ['js'],
        }
        
        # Reverse mapping
        self.reverse_synonyms = {}
        for term, syns in self.synonyms.items():
            for syn in syns:
                if syn not in self.reverse_synonyms:
                    self.reverse_synonyms[syn] = []
                self.reverse_synonyms[syn].append(term)
    
    def expand_thesaurus(self, query: str, max_expansions: int = 3) -> str:
        """
        Espande query usando il thesaurus.
        
        Args:
            query: query originale
            max_expansions: max sinonimi per termine
            
        Returns:
            Query espansa
        """
        query_lower = query.lower()
        expanded_terms = [query_lower]  # Include originale
        
        # Cerca match nel thesaurus (prima frasi, poi singole parole)
        for term, syns in self.synonyms.items():
            if term in query_lower:
                expanded_terms.extend(syns[:max_expansions])
        
        # Cerca anche in reverse
        for term, originals in self.reverse_synonyms.items():
            if term in query_lower:
                expanded_terms.extend(originals[:max_expansions])
        
        # Rimuovi duplicati mantenendo ordine
        seen = set()
        unique = []
        for t in expanded_terms:
            if t not in seen:
                seen.add(t)
                unique.append(t)
        
        return ' '.join(unique)
    
    def expand_with_corpus(self, query: str, corpus_vectorizer: TfidfVectorizer,
                           corpus_matrix, n_terms: int = 5) -> str:
        """
        Espande query con termini correlati dal corpus.
        
        Trova termini che co-occorrono frequentemente con
        i termini della query nel corpus.
        """
        # Trasforma query
        query_vec = corpus_vectorizer.transform([query])
        
        # Trova documenti più simili
        similarities = cosine_similarity(query_vec, corpus_matrix)[0]
        top_doc_idx = similarities.argsort()[-3:][::-1]  # Top 3 docs
        
        # Estrai termini importanti da quei documenti
        feature_names = corpus_vectorizer.get_feature_names_out()
        expansion_terms = set()
        
        for idx in top_doc_idx:
            doc_vec = corpus_matrix[idx].toarray()[0]
            top_term_indices = doc_vec.argsort()[-n_terms:][::-1]
            for term_idx in top_term_indices:
                if doc_vec[term_idx] > 0:
                    expansion_terms.add(feature_names[term_idx])
        
        # Combina con query originale
        expanded = query + ' ' + ' '.join(expansion_terms)
        return expanded

# Test
print("="*60)
print("DEMO 4: QUERY EXPANSION")
print("="*60)

expander = QueryExpander()

test_queries = [
    'machine learning python',
    'fix database error',
    'analyze big data fast'
]

print("\n📝 Espansione con Thesaurus:")
print("-" * 50)

for query in test_queries:
    expanded = expander.expand_thesaurus(query)
    print(f"\nOriginale: '{query}'")
    print(f"Espansa:   '{expanded}'")

# Test corpus-based expansion
print(f"\n{'='*60}")
print("📝 Espansione basata su Corpus:")
print("-" * 50)

# Usa l'engine creato prima
expanded_corpus = expander.expand_with_corpus(
    'machine learning', 
    engine.vectorizer, 
    engine.doc_vectors,
    n_terms=5
)
print(f"\nOriginale: 'machine learning'")
print(f"Espansa:   '{expanded_corpus}'")

# Confronto risultati
print(f"\n{'='*60}")
print("📊 Confronto Ricerca Originale vs Espansa")
print("-" * 50)

query_orig = 'machine learning'
query_exp = expander.expand_thesaurus(query_orig)

results_orig = engine.search(query_orig, top_k=3)
results_exp = engine.search(query_exp, top_k=3)

print(f"\nQuery originale: '{query_orig}'")
for r in results_orig:
    print(f"   {r.doc_id}: {r.score:.4f}")

print(f"\nQuery espansa: '{query_exp}'")
for r in results_exp:
    print(f"   {r.doc_id}: {r.score:.4f}")

In [ ]:
# ============================================================
# DEMO 5: SEARCH ENGINE COMPLETO
# ============================================================

class AdvancedSearchEngine:
    """
    Search engine completo con:
    - Indicizzazione TF-IDF
    - Query expansion
    - Ranking
    - Metriche di valutazione
    """
    
    def __init__(self):
        self.engine = TFIDFSearchEngine(max_features=3000)
        self.expander = QueryExpander()
        self.metrics = IRMetrics()
        self.search_history = []  # Log ricerche
    
    def index(self, documents: Dict[str, str]):
        """Indicizza documenti."""
        self.engine.index(documents)
    
    def search(self, query: str, expand: bool = False, top_k: int = 5) -> Dict:
        """
        Esegue ricerca con opzioni avanzate.
        
        Args:
            query: testo query
            expand: se True, espande la query
            top_k: numero risultati
            
        Returns:
            Dict con risultati e metadati
        """
        # Espansione opzionale
        processed_query = query
        if expand:
            processed_query = self.expander.expand_thesaurus(query)
        
        # Ricerca
        results = self.engine.search(processed_query, top_k)
        
        # Log
        search_log = {
            'original_query': query,
            'processed_query': processed_query,
            'expansion_used': expand,
            'num_results': len(results),
            'results': [{'doc_id': r.doc_id, 'score': r.score} for r in results]
        }
        self.search_history.append(search_log)
        
        return {
            'query': processed_query,
            'results': results,
            'expanded': expand
        }
    
    def evaluate(self, test_queries: Dict[str, set], expand: bool = False) -> Dict:
        """
        Valuta performance su un set di query con ground truth.
        
        Args:
            test_queries: {query: set di doc_id rilevanti}
            expand: se usare query expansion
            
        Returns:
            Dict con metriche aggregate
        """
        all_results = []
        query_metrics = []
        
        for query, relevant in test_queries.items():
            # Esegui ricerca
            search_result = self.search(query, expand=expand, top_k=10)
            retrieved = [r.doc_id for r in search_result['results']]
            
            # Calcola metriche per questa query
            p = self.metrics.precision(retrieved, relevant)
            r = self.metrics.recall(retrieved, relevant)
            f1 = self.metrics.f1_score(p, r)
            ap = self.metrics.average_precision(retrieved, relevant)
            
            query_metrics.append({
                'query': query,
                'precision': p,
                'recall': r,
                'f1': f1,
                'ap': ap
            })
            
            all_results.append((retrieved, relevant))
        
        # Metriche aggregate
        map_score = self.metrics.mean_average_precision(all_results)
        avg_p = np.mean([m['precision'] for m in query_metrics])
        avg_r = np.mean([m['recall'] for m in query_metrics])
        avg_f1 = np.mean([m['f1'] for m in query_metrics])
        
        return {
            'per_query': query_metrics,
            'aggregate': {
                'MAP': map_score,
                'avg_precision': avg_p,
                'avg_recall': avg_r,
                'avg_f1': avg_f1
            }
        }

# Test completo
print("="*60)
print("DEMO 5: SEARCH ENGINE COMPLETO")
print("="*60)

# Crea engine
adv_engine = AdvancedSearchEngine()
adv_engine.index(articles)  # Usa articles definito prima

# Test ricerca
print("\n🔍 Ricerca normale:")
result = adv_engine.search('machine learning', expand=False, top_k=3)
for r in result['results']:
    print(f"   {r.doc_id}: {r.score:.4f}")

print("\n🔍 Ricerca con expansion:")
result_exp = adv_engine.search('machine learning', expand=True, top_k=3)
print(f"   Query espansa: {result_exp['query']}")
for r in result_exp['results']:
    print(f"   {r.doc_id}: {r.score:.4f}")

# Valutazione
print(f"\n{'='*60}")
print("VALUTAZIONE SISTEMA")
print("="*60)

# Ground truth per test
test_queries = {
    'python programming': {'art1'},
    'machine learning': {'art1', 'art2', 'art7'},
    'database query': {'art3'},
    'data analysis visualization': {'art1', 'art5'},
}

# Valuta senza expansion
print("\n📊 Valutazione SENZA query expansion:")
eval_no_exp = adv_engine.evaluate(test_queries, expand=False)
print(f"   MAP: {eval_no_exp['aggregate']['MAP']:.4f}")
print(f"   Avg Precision: {eval_no_exp['aggregate']['avg_precision']:.4f}")
print(f"   Avg Recall: {eval_no_exp['aggregate']['avg_recall']:.4f}")

# Valuta con expansion
print("\n📊 Valutazione CON query expansion:")
eval_exp = adv_engine.evaluate(test_queries, expand=True)
print(f"   MAP: {eval_exp['aggregate']['MAP']:.4f}")
print(f"   Avg Precision: {eval_exp['aggregate']['avg_precision']:.4f}")
print(f"   Avg Recall: {eval_exp['aggregate']['avg_recall']:.4f}")

# Confronto
print("\n📈 Confronto:")
map_diff = eval_exp['aggregate']['MAP'] - eval_no_exp['aggregate']['MAP']
print(f"   Δ MAP con expansion: {map_diff:+.4f}")

---
# 4. Esercizi Svolti

---
## Esercizio 1: Motore di Ricerca per Documentazione Tecnica

**Obiettivo:** Costruire un motore di ricerca per documentazione tecnica con supporto per ricerca esatta e fuzzy, snippet highlighting e metriche di valutazione.

In [ ]:
# ============================================================
# ESERCIZIO 1 - SOLUZIONE
# Motore di Ricerca per Documentazione Tecnica
# ============================================================

class TechDocsSearchEngine:
    """
    Search engine specializzato per documentazione tecnica.
    
    Features:
    - Ricerca full-text con TF-IDF
    - Snippet extraction con highlighting
    - Ricerca per sezione (API, Tutorial, FAQ)
    - Metriche di performance
    """
    
    def __init__(self):
        self.vectorizer = TfidfVectorizer(
            max_features=5000,
            ngram_range=(1, 3),  # Fino a trigrammi per frasi tecniche
            stop_words='english'
        )
        self.doc_matrix = None
        self.documents = {}
        self.doc_metadata = {}  # Metadati aggiuntivi
        self.doc_ids = []
    
    def add_document(self, doc_id: str, content: str, metadata: Dict = None):
        """
        Aggiunge documento con metadati.
        
        Args:
            doc_id: identificativo documento
            content: testo del documento
            metadata: {section, tags, author, ...}
        """
        self.documents[doc_id] = content
        self.doc_metadata[doc_id] = metadata or {}
    
    def build_index(self):
        """Costruisce l'indice TF-IDF."""
        self.doc_ids = list(self.documents.keys())
        texts = [self.documents[doc_id] for doc_id in self.doc_ids]
        self.doc_matrix = self.vectorizer.fit_transform(texts)
        print(f"Indice costruito: {len(self.doc_ids)} documenti, {len(self.vectorizer.vocabulary_)} termini")
    
    def search(self, query: str, section: str = None, top_k: int = 5) -> List[Dict]:
        """
        Esegue ricerca con filtri opzionali.
        
        Args:
            query: testo query
            section: filtra per sezione (opzionale)
            top_k: numero risultati
        """
        # Trasforma query
        query_vec = self.vectorizer.transform([query])
        
        # Calcola similarità
        similarities = cosine_similarity(query_vec, self.doc_matrix)[0]
        
        # Ordina
        ranked = [(self.doc_ids[i], similarities[i]) for i in range(len(self.doc_ids))]
        ranked.sort(key=lambda x: x[1], reverse=True)
        
        # Filtra per sezione se specificato
        if section:
            ranked = [(doc_id, score) for doc_id, score in ranked
                      if self.doc_metadata.get(doc_id, {}).get('section') == section]
        
        # Costruisci risultati
        results = []
        for doc_id, score in ranked[:top_k]:
            if score > 0:
                results.append({
                    'doc_id': doc_id,
                    'score': score,
                    'snippet': self._extract_snippet(doc_id, query),
                    'metadata': self.doc_metadata.get(doc_id, {})
                })
        
        return results
    
    def _extract_snippet(self, doc_id: str, query: str, window: int = 100) -> str:
        """Estrae snippet con contesto attorno ai match."""
        text = self.documents[doc_id]
        query_terms = query.lower().split()
        
        # Trova prima occorrenza di un termine query
        text_lower = text.lower()
        best_pos = len(text)  # Default: inizio
        
        for term in query_terms:
            pos = text_lower.find(term)
            if pos != -1 and pos < best_pos:
                best_pos = pos
        
        # Estrai finestra
        start = max(0, best_pos - window // 2)
        end = min(len(text), best_pos + window)
        
        snippet = text[start:end]
        if start > 0:
            snippet = '...' + snippet
        if end < len(text):
            snippet = snippet + '...'
        
        # Highlight (semplice: uppercase)
        for term in query_terms:
            snippet = re.sub(
                rf'\b({term})\b',
                r'**\1**',
                snippet,
                flags=re.IGNORECASE
            )
        
        return snippet

# Dataset documentazione tecnica
tech_docs = {
    'api_auth': {
        'content': """
        API Authentication Guide
        
        All API requests require authentication using an API key.
        Include the API key in the header: Authorization: Bearer YOUR_API_KEY
        
        To obtain an API key, register at the developer portal.
        Keys have rate limits of 1000 requests per hour.
        """,
        'metadata': {'section': 'API', 'tags': ['authentication', 'security']}
    },
    'api_endpoints': {
        'content': """
        API Endpoints Reference
        
        GET /users - List all users
        POST /users - Create new user
        GET /users/{id} - Get user by ID
        PUT /users/{id} - Update user
        DELETE /users/{id} - Delete user
        
        All endpoints return JSON responses.
        """,
        'metadata': {'section': 'API', 'tags': ['endpoints', 'reference']}
    },
    'tutorial_quickstart': {
        'content': """
        Quickstart Tutorial
        
        Step 1: Install the SDK using pip install our-sdk
        Step 2: Import the library and initialize with your API key
        Step 3: Make your first API call
        
        Example code:
        from our_sdk import Client
        client = Client(api_key='YOUR_KEY')
        users = client.get_users()
        """,
        'metadata': {'section': 'Tutorial', 'tags': ['quickstart', 'beginner']}
    },
    'tutorial_advanced': {
        'content': """
        Advanced Usage Tutorial
        
        Batch operations allow processing multiple items efficiently.
        Use pagination for large result sets.
        Implement retry logic for handling rate limits.
        
        Advanced error handling with custom exceptions.
        """,
        'metadata': {'section': 'Tutorial', 'tags': ['advanced', 'batch']}
    },
    'faq_errors': {
        'content': """
        FAQ: Common Errors
        
        Q: Why do I get 401 Unauthorized?
        A: Your API key is invalid or expired. Check the developer portal.
        
        Q: What does 429 Too Many Requests mean?
        A: You exceeded the rate limit. Wait and retry.
        
        Q: How to fix 500 Internal Server Error?
        A: This is a server issue. Contact support.
        """,
        'metadata': {'section': 'FAQ', 'tags': ['errors', 'troubleshooting']}
    }
}

# Crea e popola engine
print("="*60)
print("ESERCIZIO 1: SEARCH ENGINE DOCUMENTAZIONE")
print("="*60)

doc_engine = TechDocsSearchEngine()

for doc_id, doc_data in tech_docs.items():
    doc_engine.add_document(doc_id, doc_data['content'], doc_data['metadata'])

doc_engine.build_index()

# Test ricerche
print("\n" + "="*60)
print("TEST RICERCHE")
print("="*60)

# Ricerca generale
print("\n🔍 Query: 'api key authentication'")
print("-" * 50)
results = doc_engine.search('api key authentication', top_k=3)
for r in results:
    print(f"\n   [{r['doc_id']}] Score: {r['score']:.4f}")
    print(f"   Section: {r['metadata'].get('section', 'N/A')}")
    print(f"   Snippet: {r['snippet'][:100]}...")

# Ricerca filtrata per sezione
print("\n🔍 Query: 'error' (solo FAQ)")
print("-" * 50)
results_faq = doc_engine.search('error', section='FAQ', top_k=3)
for r in results_faq:
    print(f"\n   [{r['doc_id']}] Score: {r['score']:.4f}")
    print(f"   Tags: {r['metadata'].get('tags', [])}")

# Ricerca tecnica
print("\n🔍 Query: 'rate limit requests'")
print("-" * 50)
results_tech = doc_engine.search('rate limit requests', top_k=2)
for r in results_tech:
    print(f"\n   [{r['doc_id']}] Score: {r['score']:.4f}")

---
## Esercizio 2: Sistema di Raccomandazione Articoli

**Obiettivo:** Creare un sistema che raccomanda articoli simili basandosi sul contenuto testuale e traccia le metriche di engagement.

In [ ]:
# ============================================================
# ESERCIZIO 2 - SOLUZIONE
# Sistema di Raccomandazione Articoli Simili
# ============================================================

class ArticleRecommender:
    """
    Sistema di raccomandazione content-based per articoli.
    
    Features:
    - Similarità basata su TF-IDF del contenuto
    - Boost per categoria e autore
    - Tracciamento click per valutazione
    """
    
    def __init__(self):
        self.vectorizer = TfidfVectorizer(
            max_features=3000,
            stop_words='english',
            ngram_range=(1, 2)
        )
        self.articles = {}
        self.article_vectors = None
        self.article_ids = []
        self.click_log = []  # [(article_shown, article_clicked), ...]
    
    def add_articles(self, articles: Dict[str, Dict]):
        """
        Aggiunge articoli al sistema.
        
        Args:
            articles: {
                'art_id': {
                    'title': str,
                    'content': str,
                    'category': str,
                    'author': str
                }
            }
        """
        self.articles = articles
        self.article_ids = list(articles.keys())
        
        # Combina title + content per vettorizzazione
        texts = [
            f"{a['title']} {a['title']} {a['content']}"  # Title pesato 2x
            for a in articles.values()
        ]
        
        self.article_vectors = self.vectorizer.fit_transform(texts)
        print(f"Indicizzati {len(articles)} articoli")
    
    def get_similar(self, article_id: str, top_k: int = 5,
                    category_boost: float = 0.2,
                    author_boost: float = 0.1) -> List[Dict]:
        """
        Trova articoli simili a quello dato.
        
        Args:
            article_id: ID articolo di partenza
            top_k: numero raccomandazioni
            category_boost: bonus per stessa categoria
            author_boost: bonus per stesso autore
        """
        if article_id not in self.articles:
            return []
        
        # Ottieni indice
        idx = self.article_ids.index(article_id)
        source_article = self.articles[article_id]
        
        # Calcola similarità con tutti
        similarities = cosine_similarity(
            self.article_vectors[idx],
            self.article_vectors
        )[0]
        
        # Applica boost per categoria/autore
        adjusted_scores = []
        for i, art_id in enumerate(self.article_ids):
            if art_id == article_id:
                continue  # Escludi se stesso
            
            score = similarities[i]
            art = self.articles[art_id]
            
            # Boost categoria
            if art.get('category') == source_article.get('category'):
                score += category_boost
            
            # Boost autore
            if art.get('author') == source_article.get('author'):
                score += author_boost
            
            adjusted_scores.append((art_id, score, art))
        
        # Ordina e prendi top_k
        adjusted_scores.sort(key=lambda x: x[1], reverse=True)
        
        recommendations = []
        for art_id, score, art in adjusted_scores[:top_k]:
            recommendations.append({
                'article_id': art_id,
                'title': art['title'],
                'score': score,
                'category': art.get('category'),
                'reason': self._explain_recommendation(source_article, art, score)
            })
        
        return recommendations
    
    def _explain_recommendation(self, source: Dict, target: Dict, score: float) -> str:
        """Genera spiegazione della raccomandazione."""
        reasons = []
        
        if source.get('category') == target.get('category'):
            reasons.append("stessa categoria")
        if source.get('author') == target.get('author'):
            reasons.append("stesso autore")
        
        if reasons:
            return f"Contenuto simile + {', '.join(reasons)}"
        return "Contenuto simile"
    
    def log_click(self, shown_article: str, clicked_article: str):
        """Registra click su raccomandazione."""
        self.click_log.append((shown_article, clicked_article))
    
    def calculate_ctr(self, article_id: str = None) -> Dict:
        """
        Calcola Click-Through Rate.
        
        CTR = clicks / impressions
        """
        if not self.click_log:
            return {'ctr': 0.0, 'clicks': 0, 'impressions': 0}
        
        impressions = len(self.click_log)
        clicks = len([1 for _, clicked in self.click_log if clicked])
        
        if article_id:
            impressions = len([1 for shown, _ in self.click_log if shown == article_id])
            clicks = len([1 for shown, clicked in self.click_log 
                          if shown == article_id and clicked])
        
        return {
            'ctr': clicks / impressions if impressions > 0 else 0.0,
            'clicks': clicks,
            'impressions': impressions
        }

# Dataset articoli
news_articles = {
    'tech1': {
        'title': 'Introduction to Machine Learning',
        'content': 'Machine learning is transforming how we analyze data. Neural networks and deep learning algorithms can recognize patterns in large datasets.',
        'category': 'Technology',
        'author': 'John Smith'
    },
    'tech2': {
        'title': 'Python for Data Science',
        'content': 'Python has become the go-to language for data science. Libraries like pandas and scikit-learn make data analysis accessible.',
        'category': 'Technology',
        'author': 'Jane Doe'
    },
    'tech3': {
        'title': 'Deep Learning Applications',
        'content': 'Deep learning neural networks are used in image recognition, natural language processing, and autonomous vehicles.',
        'category': 'Technology',
        'author': 'John Smith'
    },
    'biz1': {
        'title': 'Business Analytics Trends',
        'content': 'Companies are investing in data analytics to drive decisions. Business intelligence tools help visualize key metrics.',
        'category': 'Business',
        'author': 'Jane Doe'
    },
    'biz2': {
        'title': 'Digital Transformation Strategy',
        'content': 'Digital transformation requires both technology and organizational change. Data-driven culture is essential for success.',
        'category': 'Business',
        'author': 'Mike Johnson'
    },
    'sci1': {
        'title': 'Data in Scientific Research',
        'content': 'Scientific research generates vast amounts of data. Machine learning helps analyze experimental results and discover patterns.',
        'category': 'Science',
        'author': 'John Smith'
    }
}

# Test
print("="*60)
print("ESERCIZIO 2: RACCOMANDAZIONE ARTICOLI")
print("="*60)

recommender = ArticleRecommender()
recommender.add_articles(news_articles)

# Test raccomandazioni
print("\n📰 Articolo corrente: 'Introduction to Machine Learning' (tech1)")
print("-" * 50)
recs = recommender.get_similar('tech1', top_k=4)

for i, rec in enumerate(recs, 1):
    print(f"\n   {i}. {rec['title']}")
    print(f"      Score: {rec['score']:.4f}")
    print(f"      Categoria: {rec['category']}")
    print(f"      Motivo: {rec['reason']}")

# Simula click
recommender.log_click('tech1', 'tech3')  # Click
recommender.log_click('tech1', 'sci1')   # Click
recommender.log_click('tech1', None)     # No click
recommender.log_click('tech1', 'tech2')  # Click

# Metriche
print(f"\n{'='*60}")
print("METRICHE CTR")
print("="*60)
ctr = recommender.calculate_ctr()
print(f"\n📊 CTR globale: {ctr['ctr']:.1%}")
print(f"   Clicks: {ctr['clicks']}")
print(f"   Impressions: {ctr['impressions']}")

---
## Esercizio 3: Benchmark di Sistemi IR

**Obiettivo:** Creare un framework per confrontare diverse configurazioni di un sistema IR e visualizzare le metriche comparative.

In [ ]:
# ============================================================
# ESERCIZIO 3 - SOLUZIONE
# Benchmark Framework per Sistemi IR
# ============================================================

class IRBenchmark:
    """
    Framework per benchmarking di configurazioni IR.
    
    Confronta diverse configurazioni:
    - Con/senza stopwords
    - Unigrammi vs bigrammi vs trigrammi
    - Diverse strategie di weighting
    """
    
    def __init__(self, documents: Dict[str, str], 
                 test_queries: Dict[str, set]):
        """
        Args:
            documents: {doc_id: text}
            test_queries: {query: set di doc_id rilevanti}
        """
        self.documents = documents
        self.test_queries = test_queries
        self.results = {}
    
    def run_config(self, config_name: str, vectorizer_params: Dict) -> Dict:
        """
        Esegue benchmark per una configurazione.
        
        Args:
            config_name: nome identificativo
            vectorizer_params: parametri per TfidfVectorizer
        """
        # Crea vectorizer con configurazione
        vectorizer = TfidfVectorizer(**vectorizer_params)
        
        # Indicizza
        doc_ids = list(self.documents.keys())
        texts = [self.documents[doc_id] for doc_id in doc_ids]
        doc_matrix = vectorizer.fit_transform(texts)
        
        # Valuta su tutte le query
        metrics_collector = IRMetrics()
        all_results = []
        
        for query, relevant in self.test_queries.items():
            # Trasforma query
            query_vec = vectorizer.transform([query])
            
            # Calcola similarità
            sims = cosine_similarity(query_vec, doc_matrix)[0]
            
            # Ranking
            ranked_indices = sims.argsort()[::-1][:10]
            retrieved = [doc_ids[i] for i in ranked_indices if sims[i] > 0]
            
            all_results.append((retrieved, relevant))
        
        # Calcola metriche aggregate
        map_score = metrics_collector.mean_average_precision(all_results)
        
        avg_p = np.mean([metrics_collector.precision(r, rel) 
                         for r, rel in all_results])
        avg_r = np.mean([metrics_collector.recall(r, rel) 
                         for r, rel in all_results])
        
        result = {
            'config_name': config_name,
            'params': vectorizer_params,
            'vocab_size': len(vectorizer.vocabulary_),
            'MAP': map_score,
            'avg_precision': avg_p,
            'avg_recall': avg_r,
            'avg_f1': metrics_collector.f1_score(avg_p, avg_r)
        }
        
        self.results[config_name] = result
        return result
    
    def compare_all(self) -> None:
        """Visualizza confronto tra tutte le configurazioni."""
        if not self.results:
            print("Nessun risultato. Eseguire run_config() prima.")
            return
        
        print("\n" + "="*80)
        print("CONFRONTO CONFIGURAZIONI IR")
        print("="*80)
        
        # Header
        print(f"\n{'Configurazione':<25} {'Vocab':>8} {'MAP':>8} {'P':>8} {'R':>8} {'F1':>8}")
        print("-" * 80)
        
        # Ordina per MAP decrescente
        sorted_configs = sorted(self.results.values(), 
                                key=lambda x: x['MAP'], reverse=True)
        
        for r in sorted_configs:
            print(f"{r['config_name']:<25} {r['vocab_size']:>8} "
                  f"{r['MAP']:>8.4f} {r['avg_precision']:>8.4f} "
                  f"{r['avg_recall']:>8.4f} {r['avg_f1']:>8.4f}")
        
        # Best config
        best = sorted_configs[0]
        print(f"\n🏆 Migliore configurazione: {best['config_name']}")
        print(f"   MAP = {best['MAP']:.4f}")
    
    def get_winner(self) -> str:
        """Restituisce nome della configurazione vincente (max MAP)."""
        if not self.results:
            return None
        return max(self.results.items(), key=lambda x: x[1]['MAP'])[0]

# Test benchmark
print("="*60)
print("ESERCIZIO 3: BENCHMARK SISTEMI IR")
print("="*60)

# Ground truth per benchmark
benchmark_queries = {
    'python data': {'art1', 'art3'},
    'machine learning': {'art1', 'art2', 'art7'},
    'sql database': {'art3'},
    'visualization charts': {'art5'},
    'natural language text': {'art6'}
}

# Crea benchmark
benchmark = IRBenchmark(articles, benchmark_queries)

# Configurazioni da testare
configs = [
    ('baseline_unigram', {
        'max_features': 1000,
        'ngram_range': (1, 1),
        'stop_words': None
    }),
    ('unigram_nostop', {
        'max_features': 1000,
        'ngram_range': (1, 1),
        'stop_words': 'english'
    }),
    ('bigram_nostop', {
        'max_features': 2000,
        'ngram_range': (1, 2),
        'stop_words': 'english'
    }),
    ('trigram_nostop', {
        'max_features': 3000,
        'ngram_range': (1, 3),
        'stop_words': 'english'
    }),
    ('bigram_sublinear', {
        'max_features': 2000,
        'ngram_range': (1, 2),
        'stop_words': 'english',
        'sublinear_tf': True
    })
]

# Esegui benchmark
print("\n📊 Esecuzione benchmark...")
for name, params in configs:
    result = benchmark.run_config(name, params)
    print(f"   ✓ {name}: MAP = {result['MAP']:.4f}")

# Visualizza confronto
benchmark.compare_all()

# Analisi dettagliata
print(f"\n{'='*60}")
print("ANALISI IMPATTO CONFIGURAZIONE")
print("="*60)

# Confronto stopwords
baseline = benchmark.results['baseline_unigram']['MAP']
nostop = benchmark.results['unigram_nostop']['MAP']
print(f"\n📌 Impatto rimozione stopwords:")
print(f"   Baseline: {baseline:.4f}")
print(f"   No stopwords: {nostop:.4f}")
print(f"   Δ MAP: {nostop - baseline:+.4f}")

# Confronto n-grams
uni = benchmark.results['unigram_nostop']['MAP']
bi = benchmark.results['bigram_nostop']['MAP']
tri = benchmark.results['trigram_nostop']['MAP']
print(f"\n📌 Impatto n-grams (con stopwords rimossi):")
print(f"   Unigram: {uni:.4f}")
print(f"   Bigram:  {bi:.4f}")
print(f"   Trigram: {tri:.4f}")

---
---
# 5. Conclusione Operativa

## Cosa abbiamo imparato

**Information Retrieval** è la disciplina fondamentale per costruire sistemi di ricerca:

| Componente | Funzione | Implementazione |
|------------|----------|-----------------|
| Indice Invertito | Ricerca efficiente | defaultdict + posizioni |
| TF-IDF | Rappresentazione vettoriale | TfidfVectorizer |
| Cosine Similarity | Ranking documenti | cosine_similarity() |
| Query Expansion | Migliorare recall | Thesaurus + corpus |
| Metriche | Valutazione | Precision, Recall, MAP |

## Pattern Architetturali

1. **Indicizzazione offline** → **Ricerca online**: separa i costi
2. **Vector Space Model**: documenti e query nello stesso spazio
3. **Ranked retrieval**: non solo match/no-match, ma score graduati

## Metriche Chiave

| Metrica | Formula | Uso |
|---------|---------|-----|
| Precision | TP / (TP + FP) | Qualità risultati |
| Recall | TP / (TP + FN) | Copertura |
| F1 | 2PR / (P+R) | Bilanciamento |
| Precision@K | Prec nei primi K | Top results |
| MAP | Media delle AP | Confronto sistemi |

## Quando Usare IR Classico

✅ **Ideale per:**
- Ricerca testuale keyword-based
- Matching esatto importante
- Interpretabilità richiesta
- Dataset medio-piccoli (< milioni docs)

⚠️ **Limitazioni:**
- Vocabulary mismatch (sinonimi non catturati)
- Nessuna comprensione semantica
- Per semantica: considera embeddings (Lesson 37+)

---
# 📋 BIGNAMI - Information Retrieval

## Architettura Base

```
                INDEXING                          RETRIEVAL
                   │                                  │
DOCUMENTI ──▶ Preprocess ──▶ TF-IDF ──▶ INDEX    QUERY ──▶ Preprocess ──▶ TF-IDF
                                          │                                  │
                                          └──────────┬──────────────────────┘
                                                     │
                                                     ▼
                                               COSINE SIM ──▶ RANKED DOCS
```

## Template Indice Invertito

```python
from collections import defaultdict

class InvertedIndex:
    def __init__(self):
        self.index = defaultdict(lambda: defaultdict(list))
        self.documents = {}
    
    def add_document(self, doc_id, text):
        self.documents[doc_id] = text
        tokens = text.lower().split()
        for pos, token in enumerate(tokens):
            self.index[token][doc_id].append(pos)
    
    def search(self, term):
        return dict(self.index[term.lower()])
```

## Template Search Engine TF-IDF

```python
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

class SearchEngine:
    def __init__(self):
        self.vectorizer = TfidfVectorizer(
            max_features=5000,
            stop_words='english',
            ngram_range=(1, 2)
        )
        self.doc_matrix = None
        self.doc_ids = []
    
    def index(self, documents):
        self.doc_ids = list(documents.keys())
        texts = list(documents.values())
        self.doc_matrix = self.vectorizer.fit_transform(texts)
    
    def search(self, query, top_k=5):
        query_vec = self.vectorizer.transform([query])
        sims = cosine_similarity(query_vec, self.doc_matrix)[0]
        ranked = sims.argsort()[::-1][:top_k]
        return [(self.doc_ids[i], sims[i]) for i in ranked if sims[i] > 0]
```

## Formule Metriche

| Metrica | Formula |
|---------|---------|
| Precision | `len(retrieved & relevant) / len(retrieved)` |
| Recall | `len(retrieved & relevant) / len(relevant)` |
| F1 | `2 * P * R / (P + R)` |
| P@K | `precision(retrieved[:k], relevant)` |
| AP | `sum(P@k * rel(k)) / len(relevant)` |
| MAP | `mean([AP(q) for q in queries])` |

## Template Metriche

```python
def precision(retrieved, relevant):
    if not retrieved:
        return 0.0
    return len(set(retrieved) & relevant) / len(retrieved)

def recall(retrieved, relevant):
    if not relevant:
        return 0.0
    return len(set(retrieved) & relevant) / len(relevant)

def average_precision(retrieved, relevant):
    precisions = []
    rel_count = 0
    for i, doc in enumerate(retrieved):
        if doc in relevant:
            rel_count += 1
            precisions.append(rel_count / (i + 1))
    return sum(precisions) / len(relevant) if relevant else 0.0
```

## Parametri TfidfVectorizer

| Parametro | Default | Uso |
|-----------|---------|-----|
| `max_features` | None | Limita vocabolario |
| `stop_words` | None | 'english' rimuove stopwords |
| `ngram_range` | (1,1) | (1,2) include bigrammi |
| `sublinear_tf` | False | True usa 1+log(tf) |
| `max_df` | 1.0 | Ignora termini troppo frequenti |
| `min_df` | 1 | Ignora termini troppo rari |

## Checklist Progetto IR

- [ ] Definire corpus documenti
- [ ] Preprocessing testo (lowercase, tokenize, stopwords)
- [ ] Scegliere rappresentazione (TF-IDF, BM25)
- [ ] Costruire indice
- [ ] Implementare ricerca con ranking
- [ ] Creare ground truth per valutazione
- [ ] Calcolare metriche (P, R, MAP)
- [ ] Ottimizzare parametri
- [ ] (Opzionale) Query expansion